In [24]:
import PySimpleGUI as sg
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import cv2
import horizon
import control
import dynamics

first_column = [
    [sg.Image(filename="", key='-IMAGE1-')],
    [sg.HorizontalSeparator()],
    [sg.Image(filename="", key='-IMAGE2-')]
]

second_column = [
    [sg.Image(filename="", key='-IMAGE3-')],
    [sg.HorizontalSeparator()],
    [sg.Image(filename="", key='-IMAGE4-')]
]

layout = [
    [
        sg.Column(first_column),
        sg.VSeparator(),
        sg.Column(second_column),
    ]
]

window = sg.Window("Viewer", layout, resizable=True,)

event, values = window.read(timeout=200)

horiz_base = cv2.imread("horizon_base.png")
horiz_outer = cv2.imread("horizon_outer.png", cv2.IMREAD_UNCHANGED)
horiz_roll = cv2.imread("horizon_roll.png", cv2.IMREAD_UNCHANGED)
horiz_pitch = cv2.imread("horizon_pitch.png", cv2.IMREAD_UNCHANGED)

horiz_pitch = cv2.resize(horiz_pitch, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
horiz_roll = cv2.resize(horiz_roll, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
horiz_base = cv2.resize(horiz_base, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)
horiz_outer = cv2.resize(horiz_outer, None, fx = 0.5, fy = 0.5, interpolation= cv2.INTER_CUBIC)

img = horizon.get(0,0, 1, horiz_base, horiz_outer, horiz_roll, horiz_pitch)



imgbytes = cv2.imencode(".png", img)[1].tobytes()
window["-IMAGE1-"].update(data=imgbytes)
window["-IMAGE2-"].update(data=imgbytes)
window["-IMAGE3-"].update(data=imgbytes)
window["-IMAGE4-"].update(data=imgbytes)


state = np.array([10, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])
control_vec = np.array([0,0,0,0])


for i in range(1000):
    event, values = window.read(timeout = 1)
    if event == sg.WIN_CLOSED:
        break
        
    state = dynamics.step(state, control_vec, 0.1)
    control_vec = control.update(control_vec)
        
    plt.scatter(np.random.rand(1,10),np.random.rand(1,10))
    plt.savefig('my_plot.png', dpi=50)
    plt.clf()
    plt.close()
    img = cv2.imread("my_plot.png")
    imgbytes = cv2.imencode(".png", img)[1].tobytes()
    window["-IMAGE1-"].update(data=imgbytes)
    window["-IMAGE2-"].update(data=imgbytes)

    pitch = state[9] * 10
    roll = state[10] * 10
    
    attitude = horizon.get(roll,-pitch, 1, horiz_base, horiz_outer, horiz_roll, horiz_pitch)
    img2 = cv2.imencode(".png", attitude)[1].tobytes()
    window["-IMAGE3-"].update(data=img2)


window.close()

In [6]:
window.close()

In [23]:
state.shape

(12,)